In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1.2.6 딥러닝으로 분류 문제 풀기

In [1]:
import pandas as pd

# 데이터 구성 및 준비하기
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/NHANES Weight and Height/data.csv')

data.head()

,Unnamed: 0,Weight (kg),Standing Height (cm),BMI(kg/m**2)
0,0,97.1,160.2,37.8
1,1,98.8,182.3,29.7
2,2,74.3,184.2,21.9
3,3,103.7,185.3,30.2
4,4,83.3,177.1,26.6


In [2]:
def BMI_Classification(BMI):
    if BMI <= 18.5:
        return 0 #'저체중'
    elif (18.5 < BMI <= 23):
        return 1 #'정상'
    elif (23 < BMI <= 25):
        return 2 #'과체중'
    elif (25 < BMI <= 30):
        return 3 #'비만'
    elif BMI > 30:
        return 4 #'고도비만'

In [3]:
data['BMI_CF'] = data['BMI(kg/m**2)'].apply(BMI_Classification)

data.head(10)

,Unnamed: 0,Weight (kg),Standing Height (cm),BMI(kg/m**2),BMI_CF
0,0,97.1,160.2,37.8,4
1,1,98.8,182.3,29.7,3
2,2,74.3,184.2,21.9,1
3,3,103.7,185.3,30.2,4
4,4,83.3,177.1,26.6,3
5,5,91.1,152.7,39.1,4
6,6,72.6,158.4,28.9,3
7,7,73.0,161.2,28.1,3
8,8,81.4,161.3,31.3,4
9,9,86.0,167.8,30.5,4


In [4]:
feature_cols = ['Weight (kg)', 'Standing Height (cm)']

label = data.pop('BMI_CF')
feature = data[feature_cols].copy()

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

tf.random.set_seed(0)

In [6]:
# 모델 생성 
model = Sequential([
    Dense(units=64, activation='relu', input_shape=[2]),
    Dense(units=32, activation='relu'),
    Dense(units=16, activation='relu'),
    Dense(units=5, activation='softmax') # Classification
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                192       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 5)                 85        
                                                                 
Total params: 2,885
Trainable params: 2,885
Non-trainable params: 0
_________________________________________________________________


In [7]:
# 모델 컴파일 
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
from sklearn.model_selection import train_test_split

# 학습과 테스트 데이터 분리 (학습 80%, 테스트 20%)
train_x, test_x, train_y, test_y = train_test_split(feature, label, test_size=0.2, random_state=0)

# 학습과 검증 데이터 분리 (학습 80%, 검증 20%)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

print('train dataset :', train_x.shape, train_y.shape)
print('validation dataset :', val_x.shape, val_y.shape)
print('test dataset :', test_x.shape, test_y.shape)

train dataset : (5368, 2) (5368,)
validation dataset : (1342, 2) (1342,)
test dataset : (1678, 2) (1678,)


In [9]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping
early_stopping = EarlyStopping(patience = 20)

history = model.fit(train_x, train_y,
                    validation_data=(val_x, val_y),
                    epochs=1000,
                    callbacks=[early_stopping])

Epoch 1/1000
168/168 [==============================] - 1s 4ms/step - loss: 1.4238 - accuracy: 0.6613 - val_loss: 0.5791 - val_accuracy: 0.7727
Epoch 2/1000
168/168 [==============================] - 0s 3ms/step - loss: 0.5552 - accuracy: 0.7697 - val_loss: 0.4777 - val_accuracy: 0.7973
Epoch 3/1000
168/168 [==============================] - 0s 3ms/step - loss: 0.5096 - accuracy: 0.7828 - val_loss: 0.4922 - val_accuracy: 0.7899
Epoch 4/1000
168/168 [==============================] - 0s 3ms/step - loss: 0.4990 - accuracy: 0.7833 - val_loss: 0.4583 - val_accuracy: 0.7973
Epoch 5/1000
168/168 [==============================] - 0s 3ms/step - loss: 0.4962 - accuracy: 0.7807 - val_loss: 0.4876 - val_accuracy: 0.7876
Epoch 6/1000
168/168 [==============================] - 0s 3ms/step - loss: 0.4833 - accuracy: 0.7841 - val_loss: 0.4492 - val_accuracy: 0.7988
Epoch 7/1000
168/168 [==============================] - 0s 3ms/step - loss: 0.4875 - accuracy: 0.7873 - val_loss: 0.4575 - val_accuracy:

In [10]:
import plotly.graph_objects as go

fig = go.Figure( )
fig.add_trace(go.Scattergl(y=history.history['loss'], name='loss')) 
fig.add_trace(go.Scattergl(y=history.history['val_loss'], name='val_loss')) 
fig.update_layout(title="<b>Loss of Model</b>", xaxis_title='Epoch', yaxis_title='Loss', template='seaborn')
fig.show( )

In [11]:
x_test = [(91.1, 152.7)]
#y_test = [39.1] = 4

print(model.predict(x_test))

1/1 [==============================] - 0s 121ms/step
[[0.0000000e+00 5.6273304e-25 8.0028134e-13 5.6491781e-06 9.9999440e-01]]


In [12]:
import numpy as np

np.argmax(model.predict(x_test))

1/1 [==============================] - 0s 38ms/step


4

In [13]:
model.evaluate(test_x, test_y)

53/53 [==============================] - 0s 2ms/step - loss: 0.0987 - accuracy: 0.9559


[0.09869915246963501, 0.9558998942375183]